In [1]:
import polars as pl
from pathlib import Path
import datetime

In [3]:
files_dir = Path("../files")  # Dossier au même niveau que le répertoire actuel

In [5]:
# Charger le CSV avec les bonnes colonnes
df = pl.read_csv(
    files_dir/ "nom_prenoms_lower_trim.csv",
    separator=",",  # Ajuster selon le séparateur réel
    new_columns=["nom", "prenoms", "sexe", "age"],
    dtypes={
        "nom": pl.Utf8,
        "prenoms": pl.Utf8,
        "sexe": pl.Categorical,  # Optimisation mémoire
        "age": pl.UInt8
    }
)

C:\Users\USER\AppData\Local\Temp\ipykernel_9960\1450877043.py:2: DeprecationWarning: The argument `dtypes` for `read_csv` is deprecated. It has been renamed to `schema_overrides`.
  df = pl.read_csv(


In [4]:
df

shape: (27_168_543, 4)
┌──────────┬───────────┬──────────┬─────┐
│ nom      ┆ prenoms   ┆ sexe     ┆ age │
│ ---      ┆ ---       ┆ ---      ┆ --- │
│ str      ┆ str       ┆ cat      ┆ u8  │
╞══════════╪═══════════╪══════════╪═════╡
│ lago     ┆ serge     ┆ Masculin ┆ 43  │
│ lago     ┆ renaud    ┆ Masculin ┆ 40  │
│ lago     ┆ therese   ┆ Féminin  ┆ 68  │
│ bode     ┆ charleine ┆ Féminin  ┆ 25  │
│ dina     ┆ dina      ┆ Féminin  ┆ 27  │
│ …        ┆ …         ┆ …        ┆ …   │
│ diallo   ┆ maiga     ┆ Masculin ┆ 46  │
│ diallo   ┆ amina     ┆ Féminin  ┆ 28  │
│ diallo   ┆ ramata    ┆ Féminin  ┆ 21  │
│ ouedrago ┆ salif     ┆ Masculin ┆ 24  │
│ maiga    ┆ mariam    ┆ Féminin  ┆ 22  │
└──────────┴───────────┴──────────┴─────┘

In [5]:
# 2. Calculer l'année de naissance (ex: 2023 - age)
current_year = 2021
df = df.with_columns(
    annee_naissance = current_year - pl.col("age")
)

In [6]:
df

shape: (27_168_543, 5)
┌──────────┬───────────┬──────────┬─────┬─────────────────┐
│ nom      ┆ prenoms   ┆ sexe     ┆ age ┆ annee_naissance │
│ ---      ┆ ---       ┆ ---      ┆ --- ┆ ---             │
│ str      ┆ str       ┆ cat      ┆ u8  ┆ u16             │
╞══════════╪═══════════╪══════════╪═════╪═════════════════╡
│ lago     ┆ serge     ┆ Masculin ┆ 43  ┆ 1978            │
│ lago     ┆ renaud    ┆ Masculin ┆ 40  ┆ 1981            │
│ lago     ┆ therese   ┆ Féminin  ┆ 68  ┆ 1953            │
│ bode     ┆ charleine ┆ Féminin  ┆ 25  ┆ 1996            │
│ dina     ┆ dina      ┆ Féminin  ┆ 27  ┆ 1994            │
│ …        ┆ …         ┆ …        ┆ …   ┆ …               │
│ diallo   ┆ maiga     ┆ Masculin ┆ 46  ┆ 1975            │
│ diallo   ┆ amina     ┆ Féminin  ┆ 28  ┆ 1993            │
│ diallo   ┆ ramata    ┆ Féminin  ┆ 21  ┆ 2000            │
│ ouedrago ┆ salif     ┆ Masculin ┆ 24  ┆ 1997            │
│ maiga    ┆ mariam    ┆ Féminin  ┆ 22  ┆ 1999            │
└──────────┴───────────┴──────────┴─────┴─────────────────┘

In [7]:
df_exploded = df.with_columns(
    pl.col("prenoms").str.split(" ").alias("prenoms_split")
).explode("prenoms_split").rename({"prenoms_split": "prenom_explode"})

In [8]:
df_exploded

shape: (37_488_637, 6)
┌──────────┬───────────┬──────────┬─────┬─────────────────┬────────────────┐
│ nom      ┆ prenoms   ┆ sexe     ┆ age ┆ annee_naissance ┆ prenom_explode │
│ ---      ┆ ---       ┆ ---      ┆ --- ┆ ---             ┆ ---            │
│ str      ┆ str       ┆ cat      ┆ u8  ┆ u16             ┆ str            │
╞══════════╪═══════════╪══════════╪═════╪═════════════════╪════════════════╡
│ lago     ┆ serge     ┆ Masculin ┆ 43  ┆ 1978            ┆ serge          │
│ lago     ┆ renaud    ┆ Masculin ┆ 40  ┆ 1981            ┆ renaud         │
│ lago     ┆ therese   ┆ Féminin  ┆ 68  ┆ 1953            ┆ therese        │
│ bode     ┆ charleine ┆ Féminin  ┆ 25  ┆ 1996            ┆ charleine      │
│ dina     ┆ dina      ┆ Féminin  ┆ 27  ┆ 1994            ┆ dina           │
│ …        ┆ …         ┆ …        ┆ …   ┆ …               ┆ …              │
│ diallo   ┆ maiga     ┆ Masculin ┆ 46  ┆ 1975            ┆ maiga          │
│ diallo   ┆ amina     ┆ Féminin  ┆ 28  ┆ 1993            ┆ amina          │
│ diallo   ┆ ramata    ┆ Féminin  ┆ 21  ┆ 2000            ┆ ramata         │
│ ouedrago ┆ salif     ┆ Masculin ┆ 24  ┆ 1997            ┆ salif          │
│ maiga    ┆ mariam    ┆ Féminin  ┆ 22  ┆ 1999            ┆ mariam         │
└──────────┴───────────┴──────────┴─────┴─────────────────┴────────────────┘

In [9]:
# Calcul des occurrences annuelles
yearly_stats = df_exploded.group_by(
    "prenom_explode", "annee_naissance"
).agg(
    pl.count().alias("count")
).pivot(
    values="count",
    index="prenom_explode",
    columns="annee_naissance",
    aggregate_function="sum"
).fill_null(0)

# Renommer les colonnes d'année
year_columns = [str(year) for year in range(1886, 2022)]
yearly_stats = yearly_stats.rename(
    {str(year): f"annee_{year}" for year in range(1886, 2022)}
)


C:\Users\USER\AppData\Local\Temp\ipykernel_22884\2852160622.py:5: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count().alias("count")
C:\Users\USER\AppData\Local\Temp\ipykernel_22884\2852160622.py:6: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  ).pivot(


In [10]:
yearly_stats_sorted = yearly_stats.select(sorted(yearly_stats.columns))

In [11]:
yearly_stats_sorted

shape: (1_351_302, 137)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ annee_188 ┆ annee_188 ┆ annee_188 ┆ annee_188 ┆ … ┆ annee_201 ┆ annee_202 ┆ annee_202 ┆ prenom_e │
│ 6         ┆ 7         ┆ 8         ┆ 9         ┆   ┆ 9         ┆ 0         ┆ 1         ┆ xplode   │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ u32       ┆ u32       ┆ u32       ┆ u32       ┆   ┆ u32       ┆ u32       ┆ u32       ┆ str      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 0         ┆ 0         ┆ 0         ┆ 0         ┆ … ┆ 0         ┆ 2         ┆ 1         ┆ zirimba  │
│ 0         ┆ 0         ┆ 0         ┆ 0         ┆ … ┆ 0         ┆ 0         ┆ 1         ┆ n'gonan  │
│ 0         ┆ 0         ┆ 0         ┆ 0         ┆ … ┆ 0         ┆ 0         ┆ 0         ┆ djemsse  │
│ 0         ┆ 0         ┆ 0         ┆ 0         ┆ … ┆ 1         ┆ 0         ┆ 0         ┆ marie-da │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ niele    │
│ 0         ┆ 0         ┆ 0         ┆ 0         ┆ … ┆ 296       ┆ 268       ┆ 250       ┆ karamoko │
│ …         ┆ …         ┆ …         ┆ …         ┆ … ┆ …         ┆ …         ┆ …         ┆ …        │
│ 0         ┆ 0         ┆ 0         ┆ 0         ┆ … ┆ 0         ┆ 0         ┆ 0         ┆ yataga   │
│ 0         ┆ 0         ┆ 0         ┆ 0         ┆ … ┆ 0         ┆ 0         ┆ 0         ┆ zeiza    │
│ 0         ┆ 0         ┆ 0         ┆ 0         ┆ … ┆ 0         ┆ 0         ┆ 0         ┆ illustte │
│ 0         ┆ 0         ┆ 0         ┆ 0         ┆ … ┆ 0         ┆ 1         ┆ 0         ┆ tebaragu │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ a        │
│ 0         ┆ 0         ┆ 0         ┆ 0         ┆ … ┆ 0         ┆ 0         ┆ 0         ┆ lorainr  │
└───────────┴───────────┴───────────┴───────────┴───┴───────────┴───────────┴───────────┴──────────┘

In [12]:
stats = (
    pl.concat([
        df_exploded.select(
            pl.col("prenom_explode").alias("nom"), 
            "sexe"
        ).with_columns(type=pl.lit("prenoms")),
        
        df.select(
            pl.col("nom").alias("nom"),
            "sexe"
        ).with_columns(type=pl.lit("famille"))
    ])
    .group_by("nom", "sexe", "type")
    .agg(count=pl.len())
    .pivot(
        values="count",
        index="nom",
        columns=["sexe", "type"],
        aggregate_function="sum"
    )
    .fill_null(0)
    .rename({
        '{"Féminin","famille"}': "nombre_femme_nom_famille",
        '{"Féminin","prenoms"}': "nombre_femme_prenom",
        '{"Masculin","famille"}': "nombre_homme_nom_famille",
        '{"Masculin","prenoms"}': "nombre_homme_prenom"  # Correction orthographique
    })
    
)


C:\Users\USER\AppData\Local\Temp\ipykernel_18980\2881166339.py:15: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  .pivot(


In [13]:
stats

shape: (1_868_968, 5)
┌────────────┬─────────────────────┬─────────────────────┬────────────────────┬────────────────────┐
│ nom        ┆ nombre_femme_prenom ┆ nombre_homme_nom_fa ┆ nombre_homme_preno ┆ nombre_femme_nom_f │
│ ---        ┆ ---                 ┆ mille               ┆ m                  ┆ amille             │
│ str        ┆ u32                 ┆ ---                 ┆ ---                ┆ ---                │
│            ┆                     ┆ u32                 ┆ u32                ┆ u32                │
╞════════════╪═════════════════════╪═════════════════════╪════════════════════╪════════════════════╡
│ wamanisa   ┆ 1                   ┆ 0                   ┆ 0                  ┆ 0                  │
│ ayouwoh    ┆ 0                   ┆ 1                   ┆ 0                  ┆ 0                  │
│ niamekey   ┆ 4                   ┆ 2                   ┆ 2                  ┆ 1                  │
│ koffitohma ┆ 0                   ┆ 0                   ┆ 1                  ┆ 0                  │
│ sanisssi   ┆ 0                   ┆ 1                   ┆ 0                  ┆ 0                  │
│ …          ┆ …                   ┆ …                   ┆ …                  ┆ …                  │
│ obrel      ┆ 0                   ┆ 0                   ┆ 1                  ┆ 0                  │
│ grodjie    ┆ 1                   ┆ 0                   ┆ 0                  ┆ 0                  │
│ wedraho    ┆ 0                   ┆ 2                   ┆ 0                  ┆ 0                  │
│ nonkoungou ┆ 0                   ┆ 0                   ┆ 0                  ┆ 1                  │
│ koidiamou  ┆ 0                   ┆ 2                   ┆ 0                  ┆ 0                  │
└────────────┴─────────────────────┴─────────────────────┴────────────────────┴────────────────────┘

In [25]:
prenom_unique_stats = (
    df_exploded.filter(pl.col("prenoms") == pl.col("prenom_explode"))
    .group_by("prenom_explode")
    .agg(prenom_unique=pl.len())
)

In [26]:
prenom_unique_stats

shape: (890_462, 2)
┌────────────────┬───────────────┐
│ prenom_explode ┆ prenom_unique │
│ ---            ┆ ---           │
│ str            ┆ u32           │
╞════════════════╪═══════════════╡
│ keuleya        ┆ 2             │
│ anaballe       ┆ 1             │
│ sinimin        ┆ 1             │
│ alsonce        ┆ 1             │
│ sorel          ┆ 47            │
│ …              ┆ …             │
│ koumannayougou ┆ 1             │
│ peinelle       ┆ 1             │
│ tenenanko      ┆ 1             │
│ yoryar         ┆ 1             │
│ abdoulwahabe   ┆ 20            │
└────────────────┴───────────────┘

In [27]:
final_stats = (
    stats.join(
        prenom_unique_stats,
        left_on="nom",
        right_on="prenom_explode",
        how="left"
    )
    .with_columns(
        partie_prenom=pl.col("nombre_homme_prenom") + pl.col("nombre_femme_prenom"),
        nom_famille=pl.col("nombre_homme_nom_famille") + pl.col("nombre_femme_nom_famille"),
        prenom_unique=pl.coalesce("prenom_unique", 0)
    )
    .select([
        "nom",
        "partie_prenom",
        "nom_famille",
        "prenom_unique",
        "nombre_homme_prenom",
        "nombre_homme_nom_famille",
        "nombre_femme_prenom",
        "nombre_femme_nom_famille"
    ])
)

In [ ]:
#Fusion avec les statistiques existantes
final_stats = stats.join(
    yearly_stats,
    left_on="nom",
    right_on="prenom_explode",
    how="left"
).drop("prenom_explode")

# Réordonnancement des colonnes
columns_order = (
    ["nom", "partie_prenom", "nom_famille", "prenom_unique"] + 
    [f"annee_{year}" for year in range(1960, 2022)] +
    ["nombre_homme_prenom", "nombre_homme_nom_famille", 
     "nombre_femme_prenom", "nombre_femme_nom_famille"]
)

final_stats = final_stats.select(columns_order)

In [28]:
final_stats

shape: (1_868_968, 8)
┌────────────┬────────────┬────────────┬───────────┬───────────┬───────────┬───────────┬───────────┐
│ nom        ┆ partie_pre ┆ nom_famill ┆ prenom_un ┆ nombre_ho ┆ nombre_ho ┆ nombre_fe ┆ nombre_fe │
│ ---        ┆ nom        ┆ e          ┆ ique      ┆ mme_preno ┆ mme_nom_f ┆ mme_preno ┆ mme_nom_f │
│ str        ┆ ---        ┆ ---        ┆ ---       ┆ m         ┆ amille    ┆ m         ┆ amille    │
│            ┆ u32        ┆ u32        ┆ u32       ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│            ┆            ┆            ┆           ┆ u32       ┆ u32       ┆ u32       ┆ u32       │
╞════════════╪════════════╪════════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╡
│ djeggrou   ┆ 1          ┆ 0          ┆ 1         ┆ 0         ┆ 0         ┆ 1         ┆ 0         │
│ zoumara    ┆ 12         ┆ 32         ┆ 12        ┆ 11        ┆ 17        ┆ 1         ┆ 15        │
│ faouziarou ┆ 1          ┆ 0          ┆ 1         ┆ 0         ┆ 0         ┆ 1         ┆ 0         │
│ zi         ┆            ┆            ┆           ┆           ┆           ┆           ┆           │
│ frevrin    ┆ 1          ┆ 0          ┆ 0         ┆ 1         ┆ 0         ┆ 0         ┆ 0         │
│ cadret     ┆ 3          ┆ 1          ┆ 2         ┆ 3         ┆ 1         ┆ 0         ┆ 0         │
│ …          ┆ …          ┆ …          ┆ …         ┆ …         ┆ …         ┆ …         ┆ …         │
│ yoranatere ┆ 1          ┆ 0          ┆ 0         ┆ 0         ┆ 0         ┆ 1         ┆ 0         │
│ orlelle    ┆ 1          ┆ 0          ┆ 0         ┆ 0         ┆ 0         ┆ 1         ┆ 0         │
│ sorre awa  ┆ 0          ┆ 1          ┆ 0         ┆ 0         ┆ 0         ┆ 0         ┆ 1         │
│ garassouba ┆ 1          ┆ 0          ┆ 1         ┆ 0         ┆ 0         ┆ 1         ┆ 0         │
│ liverato   ┆ 0          ┆ 1          ┆ 0         ┆ 0         ┆ 0         ┆ 0         ┆ 1         │
│ denis      ┆            ┆            ┆           ┆           ┆           ┆           ┆           │
└────────────┴────────────┴────────────┴───────────┴───────────┴───────────┴───────────┴───────────┘

In [67]:
final_stats.write_parquet(
     files_dir / "nom_prenoms_overlay_vf_lower_trim.parquet",
       compression="zstd"
)